In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install keras

In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [ ]:
voc_size=5000
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
#Data Cleaning
import pandas as pd
import numpy as np

df=pd.read_csv("/content/drive/My Drive/SPAM.csv")
df.dropna(axis=1)
X=np.array(df.iloc[:,1])
Y=np.array(df.iloc[:,0])

import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import re

ps=PorterStemmer()
corpus=[]
for i in range(len(X)):
  review=re.sub('^a-zA-Z',' ',X[i])
  review=review.lower()
  review=review.split()
  review=[ps.stem(word) for word in review if word not in set(stopwords.words('english'))]
  review=' '.join(review)
  corpus.append(review)


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer()
x=cv.fit_transform(corpus).toarray() 

In [ ]:
onehot_repr=[one_hot(words, voc_size) for words in corpus]

In [ ]:
sent_length=24
embedded_docs=pad_sequences(onehot_repr, padding='pre',maxlen=sent_length)

In [ ]:
#creating model
embedding_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))


model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
df['Category'].unique()
# Import label encoder 
from sklearn import preprocessing 

# label_encoder object knows how to understand word labels. 
label_encoder = preprocessing.LabelEncoder() 

# Encode labels in column 'species'. 
df['Category']= label_encoder.fit_transform(df['Category']) 

df['Category'].unique() 

y=np.array(df['Category'])
y

array([0, 0, 1, ..., 0, 0, 0])

In [ ]:
from sklearn.model_selection import train_test_split
X_train,Y_train,X_valid,Y_valid=train_test_split(x,y,random_state=40,test_size=0.25)

In [ ]:
model.fit(x, y, batch_size=40, validation_split=0.25, epochs=10)

Epoch 1/10
105/105 [==============================] - 1167s 11s/step - loss: 0.4217 - accuracy: 0.8567 - val_loss: 0.3911 - val_accuracy: 0.8693
Epoch 2/10
105/105 [==============================] - 1322s 13s/step - loss: 0.3982 - accuracy: 0.8648 - val_loss: 0.3880 - val_accuracy: 0.8693
Epoch 3/10
 51/105 [=============>................] - ETA: 10:47 - loss: 0.3929 - accuracy: 0.8672

In [ ]:
print(len(y))

5572


In [ ]:
import cv2


